    TODO : Run data preparation without filling in NAs at all

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle
import time

## prep 1: melt and append

In [ ]:
# load full data
trn = pd.read_csv('../input/train_ver2.csv')
tst = pd.read_csv('../input/test_ver2.csv')
labels = pd.read_csv('../input/labels.csv').astype(int)

# prepare lag data
trn_dates = ['2015-01-28','2015-02-28','2015-03-28','2015-04-28','2015-05-28']
tst_dates = ['2016-01-28','2016-02-28','2016-03-28','2016-04-28','2016-05-28']

temp = trn[trn['fecha_dato'] == '2015-06-28']['ncodpers']
trn_ncodpers = temp[(labels[trn['fecha_dato'] == '2015-06-28'].sum(axis=1) > 0).values].values.tolist()
tst_ncodpers = np.unique(tst['ncodpers']).tolist()

trn_trim = trn[trn['fecha_dato'].isin(trn_dates)]
trn_trim = trn_trim[trn_trim['ncodpers'].isin(trn_ncodpers)]
tst_trim = trn[trn['fecha_dato'].isin(tst_dates)]
tst_trim = tst_trim[tst_trim['ncodpers'].isin(tst_ncodpers)]

# melt labels for trn

fecha_dato = trn['fecha_dato']
train_index = (labels[fecha_dato == '2015-06-28'].sum(axis=1) > 0)
train_index = train_index[train_index == True]
train = trn.ix[train_index.index]
train.iloc[:,24:] = labels.ix[train_index.index]

trn_june = []
for ind, (run, row) in enumerate(train.iterrows()):
    for i in range(24):
        if row[24+i] == 1:
            temp = row[:24].values.tolist()
            temp.append(i)
            trn_june.append(temp)
            
# define and save target separately
target = pd.DataFrame(trn_june)[24].values.tolist()
print('# target shape : ({})'.format(len(target)))

# make full data set 
trn_june = pd.DataFrame(trn_june, columns=trn.columns[:25]).iloc[:,:-1]
trn = pd.concat([trn_trim, trn_june], axis=0)
tst = pd.concat([tst_trim, tst], axis=0)
print(trn.shape, tst.shape)

# save data
trn.to_csv('../input/trn_append.csv', index=False)
tst.to_csv('../input/tst_append.csv', index=False)
target.to_csv('../input/target.csv', index=False)

In [3]:
trn = pd.read_csv('../input/trn_append.csv')
tst = pd.read_csv('../input/tst_append.csv')
target = pd.DataFrame(pickle.load(open('../input/target.pkl','rb')))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,37,42,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## prep 2: Label Encode

In [ ]:
# clean data
skip_cols = ['fecha_dato','ncodpers']
target_cols = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
               'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
               'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
               'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
               'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
               'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
               'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
               'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

for col in trn.columns:
    if col in skip_cols:
        continue
    
    if col == 'ind_empleado':
        trn[col].fillna('S', inplace=True)
    elif col == 'age':
        trn[col].replace(' NA',0,inplace=True)
        trn[col] = trn[col].astype(str).astype(int)
        trn[col] = trn[col].astype(str).astype(int)
        continue
    elif col == 'fecha_alta':
        trn[col] = ((pd.to_datetime(trn['fecha_dato']) - pd.to_datetime(trn[col].fillna('2015-07-01')))/ np.timedelta64(1, 'D')).astype(int)
        tst[col] = ((pd.to_datetime(tst['fecha_dato']) - pd.to_datetime(tst[col]))/np.timedelta64(1, 'D')).astype(int)
        continue
    elif col == 'antiguedad':
        trn[col].replace('     NA',-1,inplace=True)
        trn[col] = trn[col].astype(str).astype(int)
        tst[col] = tst[col].astype(str).astype(int)
        continue
    elif col == 'ult_fec_cli_1t':
        trn[col] = ((pd.to_datetime(trn['fecha_dato']) - pd.to_datetime(trn[col].fillna('2015-06-30')))/ np.timedelta64(1, 'D')).astype(int)
        tst[col] = ((pd.to_datetime(tst['fecha_dato']) - pd.to_datetime(tst[col].fillna('2016-01-03')))/np.timedelta64(1, 'D')).astype(int)
        continue
    elif col == 'indrel_1mes':
        tst[col].replace('1','1.0',inplace=True)
        tst[col].replace('2','1.0',inplace=True)
        tst[col].replace('2.0','1.0',inplace=True)
        tst[col].replace(2.0,'1.0',inplace=True)
        tst[col].replace('3','3.0',inplace=True)
        tst[col].replace('4','3.0',inplace=True)
        tst[col].replace(4.0,'3.0',inplace=True)
        tst[col].replace('4.0','3.0',inplace=True)
        tst[col].replace('P','3.0',inplace=True)
    elif col == 'tiprel_1mes':
        tst[col].replace('N','I',inplace=True)
        tst[col].replace('R','P',inplace=True)
    elif col == 'indresi':
        trn[col].fillna('N',inplace=True)
    elif col == 'indext':
        trn[col].fillna('S',inplace=True)
    elif col == 'indfall':
        trn[col].fillna('N',inplace=True)
    elif col == 'tipodom':
        trn.drop([col], axis=1, inplace=True)
        tst.drop([col], axis=1, inplace=True)
        continue
    elif col == 'ind_actividad_cliente':
        trn[col].fillna(0.0, inplace=True)
    elif col == 'renta':
        tst[col].replace('         NA',0,inplace=True)
        trn[col].fillna(-1, inplace=True)
        tst[col].fillna(-1, inplace=True)
        trn[col] = trn[col].astype(str).astype(float).astype(int)
        tst[col] = tst[col].astype(str).astype(float).astype(int)
        continue
    elif col in target_cols:
        trn[col].fillna(0, inplace=True)
        trn[col] = trn[col].astype(int)
        tst[col].fillna(0, inplace=True)
        tst[col] = tst[col].astype(int)
        
    lb = LabelEncoder()
    lb.fit(pd.concat([trn[col].astype(str), tst[col].astype(str)], axis=0))
    trn[col] = lb.transform(trn[col].astype(str))
    tst[col] = lb.transform(tst[col].astype(str))

In [ ]:
# display column names
trn.columns

In [ ]:
# clean utility
col = 'ind_actividad_cliente'
print('='*50)
print(np.unique(trn[col].astype(str)), np.unique(tst[col].astype(str)))
print('='*50)
#lb = LabelEncoder()
#lb.fit(pd.concat([trn[col].astype(str), tst[col].astype(str)], axis=0))
#print(np.unique(lb.transform(trn[col].astype(str))), np.unique(lb.transform(tst[col].astype(str))))

In [ ]:
# check clean
for col in trn.columns:
    print('-'*50)
    print(trn[col].dtype, col)
    print(trn[col].dtype, col)

In [ ]:
trn.to_csv('../input/trn_append_lb.csv', index=False)
tst.to_csv('../input/tst_append_lb.csv', index=False)

## prep 3: append lag
    - for each ncodpers in June data, append Jan~May (48x5) to single June data

In [ ]:
trn_june = trn[trn['fecha_dato'] == '2015-06-28'].drop(target_cols, axis=1)
trn_othr = trn[trn['fecha_dato'] != '2015-06-28']
tst_june = tst[tst['fecha_dato'] == '2016-06-28'].drop(target_cols, axis=1)
tst_othr = tst[tst['fecha_dato'] != '2016-06-28']

In [ ]:
st = time.time()
drop_cols = ['fecha_dato','ncodpers']

print('# Appending trn data.. {} rows'.format(trn_june.shape[0]))
trn_append = []
for i, ncodper in enumerate(trn_june['ncodpers']):
    temp = trn_othr[trn_othr['ncodpers'] == ncodper].drop(drop_cols, axis=1)
    if temp.shape[0] == 0:
        row = ['NA']*225
    else:
        row = np.hstack([temp.shift(periods=i).iloc[-1,:] for i in range(temp.shape[0])]).tolist()
    trn_append.append(trn_june.iloc[i].drop(drop_cols).values.tolist() + row)
    
    if i % int(trn_june.shape[0]/10) == 0:
        print('# {} rows.. {} secs..'.format(i, round(time.time() - st),2))

st = time.time()
print('# Appending tst data.. {} rows'.format(tst_june.shape[0]))
tst_append = []
for i, ncodper in enumerate(tst_june['ncodpers']):
    temp = tst_othr[tst_othr['ncodpers'] == ncodper].drop(drop_cols, axis=1)
    if temp.shape[0] == 0:
        row = ['NA']*225
    else:
        row = np.hstack([temp.shift(periods=i).iloc[-1,:] for i in range(temp.shape[0])]).tolist()
    tst_append.append(tst_june.iloc[i].drop(drop_cols).values.tolist() + row)
    
    if i % int(tst_june.shape[0]/10) == 0:
        print('# {} rows.. {} secs..'.format(i, round(time.time() - st),2))

# 150 secs for trn
# 9964 secs for tst

In [ ]:
print(len(trn_append),len(tst_append))
print(len(trn_append[0]), len(tst_append[0]))

# 45619, 929615
# 246, 246

In [ ]:
colnames = trn_june.drop(drop_cols, axis=1).columns.values.tolist()
suffixes = ['_lag_one','_lag_two','_lag_thr','_lag_fou','_lag_fiv']
for suffix in suffixes:
    for col in trn_othr.drop(drop_cols, axis=1).columns.values.tolist():
        colnames.append(col+suffix)
print(len(colnames))
# 246

In [ ]:
# initialize column names
trn = pd.DataFrame(trn_append, columns=colnames)
tst = pd.DataFrame(tst_append, columns=colnames)
print('# trn : {} | tst : {}'.format(trn.shape, tst.shape))
# trn : (45619, 246) | tst : (929615, 246)

In [ ]:
# do not fillin na for now
#trn_trim.fillna(0,inplace=True)
#tst_trim.fillna(0,inplace=True)

In [ ]:
trn.to_csv('../input/train_append_lb_lag.csv', index=False)
tst.to_csv('../input/test_append_lb_lag.csv', index=False)

## Feature Engineering
    - Remove non-important features
    - add sum of targets
    - number of occurances via NA count [just do naCount]
    - avg of non-target columns

In [19]:
target_cols =  ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',\
                'ind_cder_fin_ult1', 'ind_cno_fin_ult1',  'ind_ctju_fin_ult1',\
                'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',\
                'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',\
                'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',\
                'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',\
                'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',\
                'ind_nomina_ult1',   'ind_nom_pens_ult1', 'ind_recibo_ult1']
lags = ['_lag_one','_lag_two','_lag_thr','_lag_fou','_lag_fiv']
diffs = [['fiv','fou'],['fou','thr'],['thr','two'],['two','one']]


In [ ]:
trn = pd.read_csv('../input/train_append_lb_lag.csv')
tst = pd.read_csv('../input/test_append_lb_lag.csv')

In [ ]:
# fe v1
trn['na_count'] = trn.isnull().sum(axis=1)/45
tst['na_count'] = tst.isnull().sum(axis=1)/45

for lag in lags:
    target_sum = 0
    trn['target_sum' + lag] = (trn[[col + lag for col in target_cols]].sum(axis=1))
    tst['target_sum' + lag] = (tst[[col + lag for col in target_cols]].sum(axis=1))
    
cols = ['ind_actividad_cliente', 'ult_fec_cli_1t']
for col in cols:
    trn[col + lag + '_avg'] = (trn[[col + lag for lag in lags]]).mean(axis=1)
    tst[col + lag + '_avg'] = (tst[[col + lag for lag in lags]]).mean(axis=1)

In [ ]:
# fe v2
for col in target_cols:
    trn[col + '_sum'] = (trn[[col + lag for lag in lags]].sum(axis=1))
    tst[col + '_sum'] = (tst[[col + lag for lag in lags]].sum(axis=1))
    
for diff in diffs:
    pre = diff[0]
    post = diff[1]
    trn['target_diff_'+post+'-'+pre] = trn['target_sum_lag_'+post] - trn['target_sum_lag_'+pre]
    tst['target_diff_'+post+'-'+pre] = tst['target_sum_lag_'+post] - tst['target_sum_lag_'+pre]

In [ ]:
trn.to_csv('../input/train_append_lb_lag_fe_v2.csv', index=False)
tst.to_csv('../input/test_append_lb_lag_fe_v2.csv', index=False)

In [26]:
trn = pd.read_csv('../input/train_append_lb_lag_fe_v2.csv')
tst = pd.read_csv('../input/test_append_lb_lag_fe_v2.csv')

In [28]:
for col in target_cols:
    for diff in diffs:
        pre = diff[0]
        post = diff[1]
        trn[col + '_label_lag_' + post] = trn[col + '_lag_' + post] - trn[col + '_lag_' + pre]
        tst[col + '_label_lag_' + post] = tst[col + '_lag_' + post] - tst[col + '_lag_' + pre]
        

trn['unique_target_count'] = (trn[[col + '_sum' for col in target_cols]] > 0).astype(int).sum(axis=1)
tst['unique_target_count'] = (tst[[col + '_sum' for col in target_cols]] > 0).astype(int).sum(axis=1)

In [31]:
trn.to_csv('../input/train_append_lb_lag_fe_v3.csv', index=False)
tst.to_csv('../input/test_append_lb_lag_fe_v3.csv', index=False)

In [ ]:
# ncodpers

temp_trn = pd.read_csv('../input/trn_append_lb.csv', usecols=['ncodpers','fecha_dato'])
temp_tst = pd.read_csv('../input/tst_append_lb.csv', usecols=['ncodpers','fecha_dato'])

trn_ncodpers = temp_trn[temp_trn['fecha_dato'] == '2015-06-28']['ncodpers']
tst_ncodpers = temp_tst[temp_tst['fecha_dato'] == '2016-06-28']['ncodpers']

trn_ncodpers.reset_index(drop=True).to_csv('../input/trn_ncodpers.csv', index=False)
tst_ncodpers.reset_index(drop=True).to_csv('../input/tst_ncodpers.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# correlation plot

corr_mat = trn.corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_mat, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(200, 150))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_mat, mask=mask, cmap=cmap, vmax=.3,
            square=True, xticklabels=5, yticklabels=5,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)

## Modeling

In [ ]:
import xgboost as xgb